In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key


In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Butaritari,KI,2021-07-22 21:04:42,3.0707,172.7902,83.66,75,29,16.46
1,1,San Patricio,US,2021-07-22 21:04:42,28.0170,-97.5169,89.49,50,72,14.67
2,2,Moussoro,TD,2021-07-22 21:04:43,13.6408,16.4901,81.27,64,100,7.61
3,3,Hobart,AU,2021-07-22 21:04:43,-42.8794,147.3294,51.26,68,75,4.00
4,4,Hilo,US,2021-07-22 21:02:14,19.7297,-155.0900,78.53,83,90,11.50


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [16]:
preferred_cities_df.count()

City_ID       186
City          186
Country       186
Date          186
Lat           186
Lng           186
Max Temp      186
Humidity      186
Cloudiness    186
Wind Speed    186
dtype: int64

In [30]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Butaritari,KI,83.66,3.0707,172.7902,
1,San Patricio,US,89.49,28.0170,-97.5169,
2,Moussoro,TD,81.27,13.6408,16.4901,
4,Hilo,US,78.53,19.7297,-155.0900,
6,Vaini,TO,78.96,-21.2000,-175.2000,
8,Esfarayen,IR,76.80,37.0765,57.5101,
13,Bengkulu,ID,75.02,-3.8004,102.2655,
19,Norman Wells,CA,79.79,65.2820,-126.8329,
21,Albany,US,79.84,42.6001,-73.9662,
25,Witu,KE,77.16,-2.3889,40.4382,


In [36]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [40]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    print(index)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")


0
1
2
4
6
8
13
19
21
Hotel not found... skipping.
25
Hotel not found... skipping.
27
28
42
45
48
49
52
61
62
73
74
75
76
79
80
Hotel not found... skipping.
82
88
89
90
92
94
96
97
105
109
Hotel not found... skipping.
110
112
114
115
117
119
Hotel not found... skipping.
124
129
130
132
137
138
139
140
141
143
146
147
157
159
161
164
167
169
170
172
179
181
182
184
185
195
199
202
205
206
207
212
214
217
Hotel not found... skipping.
225
Hotel not found... skipping.
231
Hotel not found... skipping.
238
239
243
245
246
250
257
262
264
267
270
272
277
279
282
Hotel not found... skipping.
289
294
296
298
299
300
304
307
310
311
322
324
325
327
331
336
337
342
344
346
349
350
352
356
359
363
365
368
370
374
376
378
380
384
387
388
391
394
396
401
405
411
413
Hotel not found... skipping.
416
417
418
419
420
421
426
428
430
431
436
445
449
454
455
466
468
472
475
481
483
486
492
493
495
497
498
499
500
501
502
503
510
511
512
513
517
521
522
526
530
543
546
550
553
556
Hotel not found... skippi

In [41]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Butaritari,KI,83.66,3.0707,172.7902,Isles Sunset Lodge
1,San Patricio,US,89.49,28.0170,-97.5169,"Motel 6 Sinton, TX"
2,Moussoro,TD,81.27,13.6408,16.4901,Pharmacie alnadja Moussoro
4,Hilo,US,78.53,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Vaini,TO,78.96,-21.2000,-175.2000,Keleti Beach Resort
8,Esfarayen,IR,76.80,37.0765,57.5101,دفتر ازدواج 65 : سردفتر ابراهیم برغمدی
13,Bengkulu,ID,75.02,-3.8004,102.2655,Grage Hotel Bengkulu
19,Norman Wells,CA,79.79,65.2820,-126.8329,Heritage Hotel
21,Albany,US,79.84,42.6001,-73.9662,
25,Witu,KE,77.16,-2.3889,40.4382,


In [42]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))